The following experiment computes k-anonymity, l-diversity and t-closeness for trajectory data.

The experiments is run on one-day worth of trip data from NYC yellow cabs.

The metrics are at first computed on raw data and visualized for different sizes of equivalence areas.
Later on, the data is anonymized and the metrics are computed on the anonymized data.
Finally, metrics are compared across raw data and anonymized data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import uuid

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
from utils import subtract_polygons, extract_geom

import json
with open("../data/nyc_districts.json") as json_file:
    nyc_districts = json.load(json_file)
nyc_districts = gpd.GeoDataFrame.from_features(nyc_districts['features'])
nyc_districts['BoroCD'] = nyc_districts['BoroCD'].astype(str)


# Define equivalence areas in NYC
penn_station_poly = [(-73.99828815110244, 40.750805448135566), (-73.996228214579, 40.75356880601609), (-73.98779534943618,
                                                                                                       40.7500251851189), (-73.98831033356704, 40.746725221537275), (-73.99828815110244, 40.750805448135566)]

penn_station = gpd.GeoSeries(Polygon(penn_station_poly))
penn_station = gpd.GeoDataFrame(
    {'geometry': penn_station, 'BoroCD': 'penn_station'})

grand_central_station_poly = [(-73.97714227694874, 40.7562458785223),
                              (-73.98053258914356, 40.7513857387191),
                              (-73.97669166583424, 40.749922750819835),
                              (-73.97325843829518, 40.75475048873074),
                              (-73.97714227694874, 40.7562458785223)]
grand_central_station = gpd.GeoSeries(Polygon(grand_central_station_poly))
grand_central_station = gpd.GeoDataFrame(
    {'geometry': grand_central_station, 'BoroCD': 'grand_central_station'})

port_authority_bus_poly = [(-73.99552889290536, 40.757026882235486), (-73.99063654366219, 40.754962609572274),
                           (-73.98904867592537, 40.75720567448926), (-73.99402685585702, 40.75915610420332), (-73.99552889290536, 40.757026882235486)]

port_authority_bus = gpd.GeoSeries(Polygon(port_authority_bus_poly))
port_authority_bus = gpd.GeoDataFrame(
    {'geometry': port_authority_bus, 'BoroCD': 'port_authority_bus'})

la_guardia_airport_poly = [(-73.86330344855388, 40.765973691740584), (-73.87875297247966, 40.77302654459011),
                           (-73.87390353858073, 40.77793384842783), (-73.85909774481853, 40.77003648640375), (-73.86330344855388, 40.765973691740584)]
la_guardia_airport = gpd.GeoSeries(Polygon(la_guardia_airport_poly))
la_guardia_airport = gpd.GeoDataFrame(
    {'geometry': la_guardia_airport, 'BoroCD': 'la_guardia_airport'})

jfk_airport_poly = [(-73.81083575955633, 40.64419826937766),(-73.7929829763532, 40.66340757736999),(-73.7644013570905, 40.64960348124331),(-73.78002254239324, 40.631627831232684),(-73.81083575955633, 40.64419826937766)]
jfk_airport = gpd.GeoSeries(Polygon(jfk_airport_poly))
jfk_airport = gpd.GeoDataFrame(
    {'geometry': jfk_airport, 'BoroCD': 'jfk_airport'})

battery_park_ferry_poly = [(-74.01721513018921, 40.71570211149159),(-74.01749407992676, 40.71375040736087),(-74.01363169894532, 40.71305103279016),(-74.01309525714234, 40.71516539855912),(-74.01721513018921, 40.71570211149159)]
battery_park_ferry = gpd.GeoSeries(Polygon(battery_park_ferry_poly))
battery_park_ferry = gpd.GeoDataFrame(
    {'geometry': battery_park_ferry, 'BoroCD': 'battery_park_ferry'})

whitehall_ferry_poly = [(-74.01369219413073, 40.700517009446806),(-74.01448612799913, 40.70320112120412),(-74.01158934226305, 40.7034613929551),(-74.01094561209948, 40.70090743242451),(-74.01369219413073, 40.700517009446806)]
whitehall_ferry = gpd.GeoSeries(Polygon(whitehall_ferry_poly))
whitehall_ferry = gpd.GeoDataFrame(
    {'geometry': whitehall_ferry, 'BoroCD': 'whitehall_ferry'})

wallstreet_ferry_poly = [(-74.00682730834231, 40.70238088314651),(-74.00894088904604, 40.704267853536116),(-74.00734229247317, 40.70532518426294),(-74.00508923690066, 40.7034138433873),(-74.00682730834231, 40.70238088314651)]
wallstreet_ferry = gpd.GeoSeries(Polygon(wallstreet_ferry_poly))
wallstreet_ferry = gpd.GeoDataFrame(
    {'geometry': wallstreet_ferry, 'BoroCD': 'wallstreet_ferry'})

midtown_ferry_poly = [(-74.00371185104896, 40.761439907892715),(-74.00519243042518, 40.75924574582725),(-74.00356164734413, 40.7586443702452),(-74.00186649124672, 40.76091981676726),(-74.00371185104896, 40.761439907892715)]
midtown_ferry = gpd.GeoSeries(Polygon(midtown_ferry_poly))
midtown_ferry = gpd.GeoDataFrame(
    {'geometry': midtown_ferry, 'BoroCD': 'midtown_ferry'})

# remove overlaps
boro104 = subtract_polygons(nyc_districts, '104', [
                            penn_station, port_authority_bus, midtown_ferry])
boro105 = subtract_polygons(nyc_districts, '105', [
                            penn_station, grand_central_station, port_authority_bus])
boro106 = subtract_polygons(nyc_districts, '106', [grand_central_station])
boro480 = subtract_polygons(nyc_districts, '480', [la_guardia_airport])
boro403 = subtract_polygons(nyc_districts, '403', [la_guardia_airport])
boro483 = subtract_polygons(nyc_districts, '483', [jfk_airport])
boro101 = subtract_polygons(nyc_districts, '101', [battery_park_ferry,whitehall_ferry,wallstreet_ferry])

nyc_districts = pd.concat([penn_station,
                           grand_central_station,
                           port_authority_bus,
                           la_guardia_airport,
                           jfk_airport,
                           battery_park_ferry,
                           whitehall_ferry,
                           wallstreet_ferry,
                           midtown_ferry,
                           boro101,
                           boro104,
                           boro105,
                           boro106,
                           boro403,
                           boro480,
                           boro483,
                           nyc_districts[~nyc_districts['BoroCD'].isin(
                               ['101', '104', '105', '106', '403', '480', '483'])]
                           ])
eq_areas = extract_geom(nyc_districts)

len(eq_areas)

In [ ]:
# plot all districts
from utils import plot_choropleth_polygon
plot_choropleth_polygon(pd.DataFrame({'cluster_id':nyc_districts.BoroCD.unique(),'cnt':range(nyc_districts.BoroCD.nunique())}),
nyc_districts,field='cnt')

# Get data

In [ ]:
taxi = pd.read_csv("../data/nyc_taxi_raw.csv.gz", compression="gzip",
                   parse_dates=['start_time', 'end_time'])

In [ ]:
# take only one day worth of data
from datetime import datetime
taxi = taxi[(taxi['start_time'] >= datetime(year=2009, month=1, day=5)) &
            (taxi['start_time'] < datetime(year=2009, month=1, day=6))]
taxi.shape

In [ ]:
# clean data
for col in taxi.columns:
    taxi[col].replace('', np.nan, inplace=True)
    taxi.dropna(subset=[col], inplace=True)
taxi['start_longitude'] = taxi['start_longitude'].astype(float)
taxi['start_latitude'] = taxi['start_latitude'].astype(float)
taxi['end_longitude'] = taxi['end_longitude'].astype(float)
taxi['end_latitude'] = taxi['end_latitude'].astype(float)
taxi = taxi[taxi['start_longitude'] != 0]
taxi = taxi[taxi['start_latitude'] != 0]
taxi = taxi[taxi['end_longitude'] != 0]
taxi = taxi[taxi['end_latitude'] != 0]
taxi = taxi[(taxi['start_longitude'] >= -180) &
            (taxi['start_longitude'] <= 180)]
taxi = taxi[(taxi['end_longitude'] >= -180) & (taxi['end_longitude'] <= 180)]
taxi = taxi[(taxi['start_latitude'] >= -90) & (taxi['start_latitude'] <= 90)]
taxi = taxi[(taxi['end_latitude'] >= -90) & (taxi['end_latitude'] <= 90)]
# add random ID
taxi["trajectory_id"] = [uuid.uuid4() for _ in range(len(taxi.index))]
# sort
taxi = taxi.sort_values(by=['trajectory_id', 'start_time'])
taxi.reset_index(inplace=True)

In [ ]:
# limit data to within a (conservatively large) NYC bbox
lonW = -74.67
lonE = -71.75
latN = 41.50
latS = 40.30
taxi = taxi[(taxi['start_longitude'] >= lonW) &
            (taxi['start_longitude'] <= lonE)]
taxi = taxi[(taxi['end_longitude'] >= lonW) & (taxi['end_longitude'] <= lonE)]
taxi = taxi[(taxi['start_latitude'] >= latS) &
            (taxi['start_latitude'] <= latN)]
taxi = taxi[(taxi['end_latitude'] >= latS) & (taxi['end_latitude'] <= latN)]

In [ ]:
# remove short trips
from datetime import timedelta
taxi['duration'] = (taxi.end_time-taxi.start_time)
taxi = taxi[taxi['duration'] > timedelta(seconds=60)].copy()

In [ ]:
# unix timestamps
taxi['start_time_unix'] = (taxi.start_time - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
taxi['end_time_unix'] = (taxi.end_time - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
taxi.shape

In [ ]:
# save, including random trip IDs
taxi.to_csv("../data/trips.csv.gz", index=False, compression='gzip')

In [ ]:
# taxi = pd.read_csv("../data/trips.csv.gz", compression='gzip', parse_dates=["start_time", "end_time"])

# Compute metrics with grid

In [ ]:
from itertools import product
params = {"spatial_threshold": [0.002, 0.005, 0.01],
          "temporal_threshold": [300, 600, 1800]}
experiment_conditions = [{k: v for k, v in zip(
    params.keys(), x)} for x in product(*params.values())]

In [ ]:
from utils import body
from multiprocessing import Pool, cpu_count
from functools import partial
nthreads = None

if (nthreads is not None) and (nthreads > 0) and (__name__ == "__main__"):
    pool = Pool(processes=min(cpu_count()-1,
                              min(nthreads, len(experiment_conditions))))
    results = pool.map(partial(body, df=taxi), experiment_conditions)
    pool.close()
    pool.join()
else:
    results = list(map(partial(body, df=taxi), experiment_conditions))
results = pd.concat(results)
len(results)

In [ ]:
results.to_csv("../data/results.csv.gz", index=False, compression='gzip')

In [ ]:
# results=pd.read_csv("../data/results.csv.gz",compression='gzip')

## count number of trips in each time window

To determine which time windows correspond to rush-hour

In [ ]:
# visualize results of the following experiment
spatial_param = 0.005  # 0^{\circ} 0' 18''
temporal_param = 600

In [ ]:
# define the grid in space and time to reference the data
from utils import define_equivalence_areas
_, bboxes = define_equivalence_areas(
    taxi, spatial_param, temporal_param, return_bbox=True)

In [ ]:
# find how many trajectories fall into each 1-h time window (across all the cells)
ret = {}
cells = results[(results.spatial_threshold == spatial_param) &
               (results.temporal_threshold == temporal_param)].set_index('cluster_id')
for h in range(0, 23):
    twin_limits = (int(pd.datetime(year=2009, month=1, day=5, hour=h).timestamp()),
           int(pd.datetime(year=2009, month=1, day=5, hour=h+1).timestamp()))
    cells_in_twin = bboxes[bboxes['time_box'].apply(lambda t: len(
        set(range(*t)).intersection(set(range(*twin_limits)))) > 0)]
    ret.update({f"{h}-{h+1}": pd.merge(cells_in_twin, cells, left_index=True,
                                       right_index=True, how='inner').shape[0]})
ret

# plot staircase

In [ ]:
import matplotlib.pyplot as plt
from utils import plot_staircase
fig = plt.figure(figsize=(15, 15))
ymax = max(results.k_anonymity.max(), results.l_diversity.max())
for i, st in enumerate(params['spatial_threshold']):
    for j, tt in enumerate(params['temporal_threshold']):
        ax = fig.add_subplot(len(set(params['spatial_threshold'])),
                             len(set(params['temporal_threshold'])),
                             len(set(params['temporal_threshold']))*i+j+1)
        if j == 0:
            ax.set_ylabel(f"Spatial threshold: {st}", size=18)
        if i == 0:
            ax.set_title(f"Temporal threshold: {int(tt/60)} mins", size=18)
        if i == len(set(params['spatial_threshold']))-1:
            ax.set_xlabel('Percentage of traces', fontsize=14)
        ax.set_ylim(0, ymax)
        ax.yaxis.tick_right()
        plot_staircase(results.query(
            f"spatial_threshold == {st} & temporal_threshold == {tt}"), ax=ax)

In [ ]:
import matplotlib.pyplot as plt
from utils import plot_tclose
fig = plt.figure(figsize=(15, 15))
tmax = results.t_closeness.max()
for i, st in enumerate(params['spatial_threshold']):
    for j, tt in enumerate(params['temporal_threshold']):
        ax = fig.add_subplot(len(set(params['spatial_threshold'])),
                             len(set(params['temporal_threshold'])),
                             len(set(params['temporal_threshold']))*i+j+1)
        if j == 0:
            ax.set_ylabel(f"Spatial threshold: {st}", size=18)
        if i == 0:
            ax.set_title(f"Temporal threshold: {int(tt/60)} mins", size=18)
        if i == len(set(params['spatial_threshold']))-1:
            ax.set_xlabel('Percentage of traces', fontsize=14)
#         ax.set_ylim(0,tmax)
        ax.set_yscale('log')
        ax.yaxis.tick_right()
        plot_tclose(results.query(
            f"spatial_threshold == {st} & temporal_threshold == {tt}"), ax=ax)

# Anonymize data

In [ ]:
from itertools import product
params_anon = {"spatial_threshold": [0.005],
               "temporal_threshold": [600],
               # anonymization parameters
               "spatial_noise": [0.001, 0.005, 0.02],
               "temporal_noise": [100, 600, 2400],
               # repetitions
               "nrep": list(range(3))}
anon_conditions = [{k: v for k, v in zip(
    params_anon.keys(), x)} for x in product(*params_anon.values())]

In [ ]:
from multiprocessing import Pool, cpu_count
from functools import partial
from utils import anon_body
nthreads = None

if (nthreads is not None) and (nthreads > 0) and (__name__ == "__main__"):
    pool = Pool(processes=min(cpu_count()-1,
                              min(nthreads, len(anon_conditions))))
    anon_results = pool.map(partial(anon_body, df=taxi), anon_conditions)
    pool.close()
    pool.join()
else:
    anon_results = list(map(partial(anon_body, df=taxi), anon_conditions))
anon_results = pd.concat(anon_results)
len(anon_results)

In [ ]:
anon_results.to_csv("../data/anon_results.csv.gz", index=False, compression='gzip')

In [ ]:
# anon_results=pd.read_csv("../data/anon_results.csv.gz",compression='gzip')

## plot heatmap

In [ ]:
# visualize results of the following experiment
spatial_param = 0.005
temporal_param = 600

In [ ]:
spatial_noise = spatial_param
temporal_noise = temporal_param
target_data = anon_results[(anon_results['spatial_threshold'] == spatial_param) &
                           (anon_results['temporal_threshold'] == temporal_param) &
                           (anon_results['spatial_noise'] == spatial_noise) &
                           (anon_results['temporal_noise'] == temporal_noise)]
reference_data = results[(results['spatial_threshold'] == spatial_param) &
                         (results['temporal_threshold'] == temporal_param)]

In [ ]:
vis_anon = scores_time_window(target_data, taxi, spatial_param, temporal_param,
                              (int(taxi.start_time.min().timestamp()),
                               int(taxi.end_time.max().timestamp())))
vis_anon['l_div_norm'] = vis_anon['l_diversity']/vis_anon['k_anonymity']

In [ ]:
from utils import plot_choropleth
plot_choropleth(vis_anon, field='k_anonymity')

In [ ]:
# vis_anon_morning=scores_time_window(target_data,taxi,spatial_param,temporal_param,time_window_morning)
# vis_anon_morning['l_div_norm']=vis_anon_morning['l_diversity']/vis_anon_morning['k_anonymity']

In [ ]:
# vis_anon_evening=scores_time_window(target_data,taxi,spatial_param,temporal_param,time_window_evening)
# vis_anon_evening['l_div_norm']=vis_anon_evening['l_diversity']/vis_anon_evening['k_anonymity']

In [ ]:
# plot_choropleth(vis_anon_morning,field='k_anonymity')

In [ ]:
# plot_choropleth(vis_anon_morning,field='l_div_norm')

In [ ]:
# plot_choropleth(vis_anon_evening,field='k_anonymity')

In [ ]:
# plot_choropleth(vis_anon_evening,field='l_div_norm')

## plot the difference in scores between raw and anon

In [ ]:
diff = pd.merge(reference_data[['cluster_id', 'l_diversity', 'k_anonymity', 't_closeness', 'spatial_threshold', 'temporal_threshold']
                               ].rename(columns={'k_anonymity': 'k_orig', 'l_diversity': 'l_orig', 't_closeness': 't_orig'}).drop_duplicates(),
                target_data[['cluster_id', 'nrep', 'l_diversity', 'k_anonymity', 't_closeness']
                            ].drop_duplicates().rename(columns={'k_anonymity': 'k_anon', 'l_diversity': 'l_anon', 't_closeness': 't_anon'}),
                on=['cluster_id'], how='outer')
diff['k_anonymity'] = diff['k_anon']-diff['k_orig']
diff['l_diversity'] = diff['l_anon']-diff['l_orig']
diff['t_closeness'] = diff['t_anon']-diff['t_orig']
diff.head()

In [ ]:
diff.groupby('cluster_id').agg(np.mean)[['k_anon', 'k_orig']].mean()

In [ ]:
vis_diff = scores_time_window(diff, taxi, spatial_param, temporal_param,
                              (int(taxi.start_time.min().timestamp()),
                               int(taxi.end_time.max().timestamp())))
vis_diff['l_div_norm'] = vis_diff['l_diversity']/vis_diff['k_anonymity']

In [ ]:
from utils import plot_choropleth
plot_choropleth(vis_diff.dropna(), field='k_anonymity', cmap='PuOr')

# Compute metrics with polygons

In [ ]:
temporal_param=3600 #1h

In [ ]:
# precompute clusters to save time
from utils import define_equivalence_areas_polygons
taxi=define_equivalence_areas_polygons(taxi, eq_areas, temporal_param)
taxi.head()

In [ ]:
# remove all traces that start and end in the same equivalence area
taxi=taxi[taxi.equivalence_area_start!=taxi.equivalence_area_end]
taxi.shape

In [ ]:
taxi.to_csv("../data/trips_clustered.csv.gz", index=False, compression='gzip')

In [ ]:
from utils import body_compute_metrics
results_polygons=body_compute_metrics(clustered_data=taxi)
results_polygons['temporal_threshold']=temporal_param

In [ ]:
results_polygons.to_csv("../data/results_polygons.csv.gz", index=False, compression='gzip')

In [ ]:
# results_polygons=pd.read_csv("../data/results_polygons.csv.gz",compression='gzip')

# plot heatmap for the whole day, hourly scores

In [ ]:
from utils import scores_time_window_polygon
vis = scores_time_window_polygon(results_polygons, taxi, eq_areas, temporal_param,
                         (int(taxi.start_time.min().timestamp()),
                          int(taxi.end_time.max().timestamp())))

In [ ]:
from utils import plot_choropleth_polygon
plot_choropleth_polygon(vis, nyc_districts, field='k_anonymity')

# plot heatmap for the rush hour times

In [ ]:
time_window_morning = (int(pd.datetime(year=2009, month=1, day=5, hour=7).timestamp()),
                       int(pd.datetime(year=2009, month=1, day=5, hour=7, minute=30).timestamp()))
time_window_evening = (int(pd.datetime(year=2009, month=1, day=5, hour=17).timestamp()),
                       int(pd.datetime(year=2009, month=1, day=5, hour=18).timestamp()))

### Morning

In [ ]:
from tqdm import tqdm
from utils import get_containing_polygon_name
def get_results_timewindow(df,time_window,eq_areas):
    tqdm.pandas(desc="Find data that fits in the time window")
    ret = df[df.progress_apply(lambda t: len(set(range(t.start_time_unix,t.end_time_unix)
                                                ).intersection(set(range(*time_window)))) > 0,axis=1)].copy()
    # build area names
    tqdm.pandas(desc="Match starting points with polygons")
    ret.loc[:,'equivalence_area_start'] = ret.progress_apply(lambda r: get_containing_polygon_name(
        r.start_latitude, r.start_longitude, eq_areas), axis=1)
    tqdm.pandas(desc="Match ending points with polygons")
    ret.loc[:,'equivalence_area_end'] = ret.progress_apply(lambda r: get_containing_polygon_name(
        r.end_latitude, r.end_longitude, eq_areas), axis=1)
    ret = ret[~((ret.equivalence_area_start.isin(["NA"])) | (ret.equivalence_area_end.isin(["NA"])))]
    # remove all traces that start and end in the same equivalence area
    ret=ret[ret.equivalence_area_start!=ret.equivalence_area_end]
    return ret

In [ ]:
data_morning = get_results_timewindow(taxi,time_window_morning,eq_areas)

In [ ]:
from utils import body_compute_metrics
results_morning=body_compute_metrics(clustered_data=data_morning)
results_morning.to_csv("../data/results_morning.csv.gz", index=False, compression='gzip')

In [ ]:
results_morning=results_morning.groupby('cluster_id').agg({'k_anonymity': np.mean, 'l_diversity': np.mean, 't_closeness': np.mean, 'l_div_norm': np.mean})

In [ ]:
results_morning=results_morning[results_morning.k_anonymity>1]

In [ ]:
from utils import plot_choropleth_polygon
plot_choropleth_polygon(results_morning, nyc_districts, field='k_anonymity')

In [ ]:
from utils import plot_choropleth_polygon
plot_choropleth_polygon(results_morning, nyc_districts, field='l_div_norm')

### evening

In [ ]:
data_evening = get_results_timewindow(taxi,time_window_evening,eq_areas)

In [ ]:
from utils import body_compute_metrics
results_evening=body_compute_metrics(clustered_data=data_evening)
results_evening.to_csv("../data/results_evening.csv.gz", index=False, compression='gzip')

In [ ]:
results_evening=results_evening.groupby('cluster_id').agg({'k_anonymity': np.mean, 'l_diversity': np.mean, 't_closeness': np.mean, 'l_div_norm': np.mean})

In [ ]:
results_evening=results_evening[results_evening.k_anonymity>1]

In [ ]:
plot_choropleth_polygon(results_evening, nyc_districts, field='k_anonymity')

In [ ]:
plot_choropleth_polygon(results_evening, nyc_districts, field='l_div_norm')

# Anonymize data

In [ ]:
from utils import add_noise_to_data
def anonymization_fct(e,df):
    df['start_time']=pd.to_datetime(df.start_time)
    df['end_time']=pd.to_datetime(df.end_time)
    ret=add_noise_to_data(e,df)
    for k, v in e.items():
        ret[k] = v
    return ret

In [ ]:
from itertools import product
params_anon = {# anonymization parameters
               "spatial_noise": [0.005], #[0.001, 0.005, 0.02],
               "temporal_noise": [600], #[100, 600, 2400],
               # repetitions
               "nrep": list(range(3))}
anon_conditions = [{k: v for k, v in zip(
    params_anon.keys(), x)} for x in product(*params_anon.values())]

In [ ]:
from multiprocessing import Pool, cpu_count
from functools import partial
nthreads = None

if (nthreads is not None) and (nthreads > 0) and (__name__ == "__main__"):
    pool = Pool(processes=min(cpu_count()-1,
                              min(nthreads, len(anon_conditions))))
    anon_data = pool.map(partial(anonymization_fct, df=taxi), anon_conditions)
    pool.close()
    pool.join()
else:
    anon_data = list(map(partial(anonymization_fct, df=taxi), anon_conditions))
anon_data = pd.concat(anon_data)
len(anon_data)

In [ ]:
anon_data.to_csv("../data/anon_data.csv.gz", index=False, compression='gzip')

In [ ]:
# anon_data=pd.read_csv("../data/anon_data.csv.gz",compression='gzip')

## Visualize results

In [ ]:
# visualize results of the following experiment
spatial_noise = 0.005
temporal_noise = 600

In [ ]:
target_data = anon_data[(anon_data['spatial_noise'] == spatial_noise) &
                           (anon_data['temporal_noise'] == temporal_noise)]

### Morning

In [ ]:
anon_data_morning = get_results_timewindow(target_data,time_window_morning,eq_areas)

In [ ]:
from utils import body_compute_metrics
anon_results_morning=[]
for r in anon_data_morning.nrep.unique():
    tmp=anon_data_morning[anon_data_morning.nrep==r]
    ret=body_compute_metrics(clustered_data=tmp)
    ret['nrep']=r
    anon_results_morning+=[ret]
anon_results_morning=pd.concat(anon_results_morning)
anon_results_morning.to_csv("../data/anon_results_morning.csv.gz", index=False, compression='gzip')

In [ ]:
anon_results_morning=anon_results_morning[anon_results_morning.k_anonymity>1]

In [ ]:
anon_results_morning=anon_results_morning.groupby('cluster_id').agg({'k_anonymity': np.mean, 'l_diversity': np.mean, 't_closeness': np.mean, 'l_div_norm': np.mean})

In [ ]:
from utils import plot_choropleth_polygon
plot_choropleth_polygon(anon_results_morning, nyc_districts, field='k_anonymity')

In [ ]:
from utils import plot_choropleth_polygon
plot_choropleth_polygon(anon_results_morning, nyc_districts, field='l_div_norm')

### evening

In [ ]:
anon_data_evening = get_results_timewindow(target_data,time_window_evening,eq_areas)

In [ ]:
from utils import body_compute_metrics
anon_results_evening=[]
for r in anon_data_evening.nrep.unique():
    tmp=anon_data_evening[anon_data_evening.nrep==r]
    ret=body_compute_metrics(clustered_data=tmp)
    ret['nrep']=r
    anon_results_evening+=[ret]
anon_results_evening=pd.concat(anon_results_evening)
anon_results_evening.to_csv("../data/anon_results_evening.csv.gz", index=False, compression='gzip')

In [ ]:
anon_results_evening=anon_results_evening[anon_results_evening.k_anonymity>1]

In [ ]:
anon_results_evening=anon_results_evening.groupby('cluster_id').agg({'k_anonymity': np.mean, 'l_diversity': np.mean, 't_closeness': np.mean, 'l_div_norm': np.mean})

In [ ]:
plot_choropleth_polygon(anon_results_evening, nyc_districts, field='k_anonymity')

In [ ]:
plot_choropleth_polygon(anon_results_evening, nyc_districts, field='l_div_norm')

## plot the difference in scores between raw and anon

In [ ]:
diff_morning = pd.merge(results_morning[['l_diversity', 'k_anonymity', 't_closeness', 'l_div_norm']
                               ].rename(columns={'k_anonymity': 'k_orig', 'l_diversity': 'l_orig', 't_closeness': 't_orig', 'l_div_norm': 'l_norm_orig'}).drop_duplicates(),
                anon_results_morning[['l_diversity', 'k_anonymity', 't_closeness', 'l_div_norm']
                            ].drop_duplicates().rename(columns={'k_anonymity': 'k_anon', 'l_diversity': 'l_anon', 't_closeness': 't_anon', 'l_div_norm': 'l_norm_anon'}),
                right_index=True, left_index=True, how='outer')
diff_morning['k_anonymity'] = diff_morning['k_anon']-diff_morning['k_orig']
diff_morning['l_diversity'] = diff_morning['l_anon']-diff_morning['l_orig']
diff_morning['t_closeness'] = diff_morning['t_anon']-diff_morning['t_orig']
diff_morning['l_div_norm'] = diff_morning['l_norm_anon']-diff_morning['l_norm_orig']
diff_morning.head()

In [ ]:
plot_choropleth_polygon(diff_morning.dropna(), nyc_districts, field='k_anonymity')

In [ ]:
plot_choropleth_polygon(diff_morning.dropna(), nyc_districts, field='l_div_norm')

In [ ]:
diff_evening = pd.merge(results_evening[['l_diversity', 'k_anonymity', 't_closeness', 'l_div_norm']
                               ].rename(columns={'k_anonymity': 'k_orig', 'l_diversity': 'l_orig', 't_closeness': 't_orig', 'l_div_norm': 'l_norm_orig'}).drop_duplicates(),
                anon_results_evening[['l_diversity', 'k_anonymity', 't_closeness', 'l_div_norm']
                            ].drop_duplicates().rename(columns={'k_anonymity': 'k_anon', 'l_diversity': 'l_anon', 't_closeness': 't_anon', 'l_div_norm': 'l_norm_anon'}),
                right_index=True, left_index=True, how='outer')
diff_evening['k_anonymity'] = diff_evening['k_anon']-diff_evening['k_orig']
diff_evening['l_diversity'] = diff_evening['l_anon']-diff_evening['l_orig']
diff_evening['t_closeness'] = diff_evening['t_anon']-diff_evening['t_orig']
diff_evening['l_div_norm'] = diff_evening['l_norm_anon']-diff_evening['l_norm_orig']
diff_evening.head()

In [ ]:
plot_choropleth_polygon(diff_evening.dropna(), nyc_districts, field='k_anonymity')

In [ ]:
plot_choropleth_polygon(diff_evening.dropna(), nyc_districts, field='l_div_norm')